<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/107_%E5%9E%8B%E3%81%A8%E3%82%AF%E3%83%A9%E3%82%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

型とクラス
==========

インスタンス化と継承
--------------------

**クラス**（class）は、型を定義する。クラスが型であるオブジェクトのことを、そのクラスの**インスタンス**（instance）と呼ぶ。インスタンスを生成することを**インスタンス化**（instantiation）という。

クラスごとに新たな名前空間が作成され、ローカルな名前空間として使われる。また、クラス定義のブロックは独自のスコープを持ち、クラスのローカルな名前空間にアクセスできる。

Python のクラス機構については、[公式チュートリアル](https://docs.python.org/ja/3/tutorial/classes.html#a-first-look-at-classes) でほぼ十分。

他の言語とクラス機構を比較すると、次のようになる。

| 言語 | 多重継承 | カプセル化 | オーバーライド | 抽象クラス | インターフェース |
|:-:|:-:|:-:|:-:|:-:|:-:|
| C++ | 〇 | 〇 | virtualのみ | 〇 | ×(多重継承と純粋仮想関数で対応) |
| Java | × | 〇 | 任意のメソッド | 〇 | 〇 |
| C# | × | 〇 | virtualのみ | 〇 | 〇 |
| JavaScript | × | 〇(ECMAScript 2022) | 任意のメソッド | × | × |
| Python | 〇 | ×(難読化は可能) | 任意のメソッド | ×(標準ライブラリで対応) | × |

組み込み関数 `issubclass(class, classinfo)` は、`class` に指定したクラスが、`classinfo` に指定したクラスの（直接または間接の）サブクラスであるかどうかチェックする。`classinfo` にクラスのタプルを渡すことができ、その中のどれか 1 つから継承しているかどうかをチェックする。

Python のクラスはすべて `object` クラスから継承している。

In [ ]:
class MyClass:
    pass

assert issubclass(MyClass, object)

Python において、クラスもオブジェクトであり、`type` のインスタンスである。つまり、組み込みのクラスもユーザー定義のクラスも型を持ち、それは `type` である。`object` クラスの型も `type` である。

In [ ]:
assert isinstance(int, type)
assert isinstance(MyClass, type)
assert isinstance(object, type)

`MyClass` のサブクラスとして新しいクラス `MySubClass` を追加したとき、`MySubClass` は `MyClass` から継承しているが、`type` のインスタンスであることには変わらない。

In [ ]:
class MySubClass(MyClass):
    pass

assert issubclass(MySubClass, MyClass)
assert isinstance(MySubClass, type)

クラスは `type` のインスタンスであるから、`type` そのものはクラスである。`type` クラスの型は `type` である。また、`type` クラスは、他のクラスと同様に `object` クラスから継承している。

In [ ]:
assert isinstance(type, type)
assert issubclass(type, object)

以上をまとめると、次の図のようになる（[POSTD の記事](https://postd.cc/pythons-objects-and-classes-a-visual-guide/)から引用）。`m` はクラス `MyClass` のインスタンス、`s` はクラス `MySubClass` のインスタンスとし、型と継承元を示すポインタを描いている。

![](https://postd.cc/wp/wp-content/uploads/2015/11/Python-objects-81.png)

メタクラス
----------

`type` のように、クラスの型となるものを**メタクラス**（meta class）という。

クラスのインスタンスがコンストラクタで生成されるように、クラスはメタクラスのインスタンス（クラスオブジェクトとも呼ばれる）としてコンストラクタで生成される。クラス定義は実行可能な文であり、クラス定義の実行はメタクラスのコンストラクタを呼び出すのとほとんど同じである。デフォルトで `type` がメタクラスに使用されるため、クラスは `type` のインスタンスとなっているのである。たとえば

``` python
class X:
    a = 1
```

は

``` python
X = type("X", (), dict(a=1))
```

とほとんど同じである。このように、`type()` 関数は、3 個の引数が渡される場合、新しいクラスオブジェクトを返す。3 個の引数は、左から順に、クラスオブジェクトが持つ次の 3 つの属性を設定する。

| 特殊属性 | 意味 |
|:--|:----|
| `__name__` | 名前 |
| `__bases__` | 基底クラスのタプル |
| `__dict__` | 属性辞書（クラスの名前空間の実装）。組み込み関数 `vars()` はこの値を返す |

`type()` 関数を使うことでクラスを動的に定義することができる。

次のコードでは、`list` を継承し、要素の文字列を空白区切りで連結するメソッド `concat()` を持つ `StrList` クラス（クラスオブジェクト）を `type()` で生成している。

In [ ]:
def concat(self):
    ret = ""
    for s in self:
        if ret:
            ret += " " + s
        else:
            ret = s
    return ret

StrList = type("StrList", (list,), {"concat": concat})

# インスタンス化
sl = StrList(("peek", "a"))
sl.append("boo")
slang = sl.concat()
print(slang)

peek a boo


クラス定義において `metaclass` キーワード引数で別のメタクラスを指定することにより、そのインスタンスが生成されるようにできる。つまり

``` python
class MyClass(metaclass=Meta):
    pass
```

は

``` python
MyClass = Meta("MyClass", (object,), {})
```

とほとんど等価である。次の例で `MyClass` と `MySubclass` は両方とも `Meta` のインスタンスである:

In [ ]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    pass

assert isinstance(MyClass, Meta)
assert isinstance(MySubclass, Meta)

クラス定義が実行されると、`metaclass(name, bases, namespace, **kwds)` が呼び出されてクラスオブジェクトが生成されるわけだが、デフォルトのメタクラス `type` か `type` を継承するメタクラスを使っているときは、`type.__new__()` メソッドが呼び出され、オブジェクト生成の主な作業が行われる。このメソッドは特別扱いされ、自動的にスタティックメソッドとなる。

``` python
type.__new__(cls, name, bases, namespace, **kwargs)
```

第 1 引数の `cls` にはメタクラスが渡される。

また、クラスオブジェクトの生成後、処理が戻される前に、`type.__init__()` メソッドが呼び出される。

``` python
type.__init__(self, name, bases, namespace, **kwargs)
```

`type.__new__()` や `type.__init__()` をオーバーライドすれば、クラス生成をカスタマイズできる。

次のコードでは、`type` を継承するメタクラス `Meta` において、`__new__()` をオーバーライドして、追加する属性やメソッドの名前にアンダースコア `'_'` が付くようにしている。また、`__init__()` をオーバーライドして、デフォルトで `say()` メソッドが提供されるようにしている。

In [ ]:
def say(self):
    print("Hello, world!")


class Meta(type):
    def __new__(cls, name, bases, namespace, **kwargs):
        concealment = dict(("_" + k, v) for k, v in namespace.items())
        return super().__new__(cls, name, bases, concealment, **kwargs)

    def __init__(self, name, bases, namespace, **kwargs):
        super().__init__(name, bases, namespace, **kwargs)
        self.say = say


class MyClass(metaclass=Meta):
    def test(self):
        print("This is a test.")


m = MyClass()
m.say()  # say() メソッドはクラス定義では定義されてなく、継承もしていないが、利用可能である
m._test()  # test() メソッドの名前は _test に変更されている
try:
    m.test()  # test という名前でメソッドを呼び出すとエラーが発生する
except AttributeError as err:
    print(f"{type(err).__name__}: {err}")

Hello, world!
This is a test.
AttributeError: 'MyClass' object has no attribute 'test'


super()
-------

先のコード例で組み込み関数 `super()` が使用されていたが、メソッドのオーバーライドを行うに際しては、基本的な処理を基底クラスのメソッドに委譲するため、`super()` を呼び出すことが基本である。

``` python
super(type, object_or_type=None)
```

この関数は、ドット `.` 構文で使われると、後ろに続くメソッドをメソッド解決順序（method resolution order; MRO）で検索し、そうして見つかったメソッドを持つ基底クラスを返す。MRO は `object_or_type` の特殊属性 `__mro__` を参照すれば知ることができる。この属性は動的で、継承の階層構造が更新されれば変化する。`super()` をメソッドの中で使うとき、`super()` の引数は省略できる。

In [ ]:
class A:
    def echo(self):
        print("A")

class B(A):
    pass

class C(B):
    def echo(self):
        print("C", end=", ")
        super().echo()  # super(C, self).echo() と同じ

c = C()
super(C, c).echo()
c.echo()
print(f"{C.__mro__=}")

A
C, A
C.__mro__=(<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


特殊メソッド
------------

`type` の基底クラスである `object` クラスも、`__new__()` や `__init__()` メソッドを持つ。これらのように、オブジェクトの振る舞いを決める、特殊な名前のメソッドが定義されている。

| メソッド | 機能 | 備考 |
|:--|:----|:----|
| `__new__(cls[, ...])` | スタティックメソッドであり、`cls` に指定したクラスの新しいイン<br />スタンスを返す。残りの引数はオブジェクトのコンストラクタの式<br />（クラスの呼び出し文）に渡される | クラスのコンストラクタ式を評価するときに呼び<br />出される |
| `__init__(self[, ...])` | インスタンスが `__new__()` によって生成された後、それが呼び出<br />し元に返される前に呼び出される。`self` は新たに生成されたイ<br />ンスタンスで、残りの引数はクラスのコンストラクタ式に渡したも<br />のである。`__new__()` がインスタンスを生成しないとき、<br />`__init__()` は呼び出されない | イニシャライザと呼ばれる |
| `__repr__(self)` | 同じ値のオブジェクトを再生成するのに使える、有効な Python 式<br />のような文字列を返す | 組み込み関数 `repr()` によって呼び出される |
| `__str__(self)` | オブジェクトを表現する文字列を返す（`object` のデフォルト実装<br />は `__repr__()` を呼び出す） | 組み込み関数 `str()`、`format()`、`print()` に<br />よって呼び出される |
| `__dir__(self)` | 属性のリストを返す | 組み込み関数 `dir()` によって呼び出される |
| ` __hash__(self)` | ハッシュ値を返す | 組み込み関数 `hash()` によって呼び出される |
| `__bool__(self)` | ブール値（`False` または `True`）を返す | 組み込み関数 `bool()` によって呼び出される |
| `__eq__(self, other)` | 等しい | 演算子 `==` によって呼び出される |
| `__ne__(self, other)` | 等しくない | 演算子 `!=` によって呼び出される |
| `__lt__(self, other)` | 小なり | 演算子 `<` によって呼び出される |
| `__le__(self, other)` | 小なりイコール | 演算子 `<=` によって呼び出される |
| `__gt__(self, other)` | 大なり | 演算子 `>` によって呼び出される |
| `__ge__(self, other)` | 大なりイコール | 演算子 `>=` によって呼び出される |
| `__getattribute__(self, name)` | 属性参照 | 演算子 `.` などによって無条件に呼び出される |
| `__setattr__(self, name, value)` | 属性参照 | 組み込み関数 `setattr()` によって呼び出される |
| `__delattr__(self, name)` | 属性参照 | 組み込み関数 `delattr()` によって呼び出される |

`object.__repr__()` は、オブジェクトのメモリ上のアドレスを示す `'<object object at 0x0000123456789ABC>'` 形式の文字列を出力する。しかし、クラスの `__repr__()` が返す文字列については、[公式ドキュメント](https://docs.python.org/ja/3/reference/datamodel.html#object.__repr__)によれば「同じ値のオブジェクトを再生成するのに使える、有効な Python 式のようなものであるべきです」とある。このため、定義したクラスのインスタンスに対して `repr()` の出力が必要なら、 `__repr__()` を定義しておく必要がある。

In [ ]:
a = object()
assert str(a) == repr(a) == a.__repr__()
a  # repr(a) と同じ

In [ ]:
class A:
    def __init__(self, name):
        self.name = name

A("hoge")

In [ ]:
class A:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"A('{self.name}')"

A("hoge")

A('hoge')

式 `x == y` では、まず `x.__eq__(y)` の呼び出しが試みられ、`__eq__()` が実装されていない場合、または、`__eq__()` が `NotImplemented` を返す場合、`y.__eq__(x)` の呼び出しが試みられる。他の `other` を引数に持つ特殊メソッドでも同様の動作となる。

In [ ]:
class X:
    def __init__(self, val):
        self.val = val

class Y:
    def __init__(self, val):
        self.val = val

    def __eq__(self, other):
        return other.val != self.val  # あえて反対の評価を返す

assert X(0) == Y(1)  # Y(1).__eq__(X(0)) が呼び出される

`==` や `!=` 演算子によって呼び出される特殊メソッドがクラスで独自に定義される可能性があることから、PEP 8 は、`None` に対しては `==` や `!=` でなく、`is` や `is not` を使うことを推奨している。

In [ ]:
a = None
assert a is None  # PEP 8 推奨
assert a == None  # PEP 8 非推奨

特定の型に属するオブジェクトが持つような特殊メソッドもある:

| メソッド | 機能 | 備考 |
|:--|:----|:----|
| `__mul__(self, other)` | 乗算 | 演算子 `*` によって呼び出される |
| `__matmul__(self, other)` | 行列積 | 演算子 `@` によって呼び出される |
| `__truediv__(self, other)` | 除算 | 演算子 `/` によって呼び出される |
| `__floordiv__(self, other)` | 整数除法 | 演算子 `//` によって呼び出される |
| `__mod__(self, other)` | 剰余 | 演算子 `%` によって呼び出される |
| `__add__(self, other)` | 加算 | 演算子 `+` によって呼び出される |
| `__sub__(self, other)` | 減算 | 演算子 `-` によって呼び出される |
| `__getitem__(self, key)` | インデックス参照 | 演算子 `[]` によって呼び出される |
| `__setitem__(self, key, value)` | インデックス参照 | 演算子 `[]` によって呼び出される |
| `__delitem__(self, key)` | インデックス参照 | 演算子 `[]` によって呼び出される |

式 `x * y` の評価では、まず `x.__mul__(y)` の呼び出しが試みられ、`__mul__()` が実装されていない場合、または、`__mul__()` が `NotImplemented` を返す場合、`y.__mul__(x)` の呼び出しが試みられる。他の `other` を引数に持つ特殊メソッドでも同様の動作となる。

式 `x[i]` の評価では `x.__getitem__(i)` の呼び出しが試みられ、 式 `x[i] = v` の評価では `x.__setitem__(i, v)` の呼び出しが試みられる。`x[i]` の削除では `__delitem__(i)` の呼び出しが試みられる。

これらの特殊メソッドを独自に定義することによって、オブジェクトの振る舞いをカスタマイズすることができる。これは、Python で演算子オーバロードを実現する方法である。なお、`__matmul__()` をサポートする組み込み型や標準ライブラリーはなく、行列積演算子 `@` は初めからサードパーティーのライブラリーで実装されることが期待されている（実際、`Numpy` で実装された）。

In [ ]:
class MyClass:
    def __init__(self, num):
        self.val = num
        self.even_num = 0
        self.odd_num = 1

    def __add__(self, other):
        if isinstance(other, MyClass):
            num = int(str(self.val) + str(other.val))
            return MyClass(num)
        else:
            return NotImplemented

    def __getitem__(self, key):
        if not isinstance(key, int):
            raise TypeError("key が整数でない")
        if key % 2 == 0:
            return self.even_num
        else:
            return self.odd_num

    def __setitem__(self, key, value):
        if not isinstance(key, int):
            raise TypeError("key が整数でない")
        if key % 2 == 0:
            self.even_num = value
        else:
            self.odd_num = value


m1 = MyClass(10)
m2 = MyClass(24)
assert (m1 + m2).val == 1024
m1[31] = 3
assert m1[1] == 3

ハッシュ可能
------------

オブジェクトが**ハッシュ可能**（hashable）であるとは、オブジェクトが特殊メソッド `__eq__()` および `__hash__()` を持ち、かつ、`__hash__()` が次の条件をみたしていることをいう。

  1. 生存期間中変わらない整数（ハッシュ値）を返す。
  2. 値が等しいハッシュ可能オブジェクト同士では、同じハッシュ値を返す。

ハッシュ可能なオブジェクトに対して組み込み関数 `hash()` を実行すると、その `__hash__()` メソッドが（存在すれば）呼び出されハッシュ値を返す。

ハッシュ可能なオブジェクトは `dict` オブジェクト（辞書）のキーや、`set` オブジェクト（集合）および `frozenset` オブジェクト（集合）の要素として使える。辞書や集合のデータ構造は内部でハッシュ値を使っているからである。なお、`set` オブジェクトはミュータブルだが、`frozenset` オブジェクトはイミュータブルである。

オブジェクトがイミュータブルであることと、ハッシュ可能であることは異なる概念であるが、Python のイミュータブルな組み込みオブジェクトは、ほとんどがハッシュ可能である。この例外は `tuple` と `frozenset` の 2 つであり、どちらもイミュータブルであるが、すべての要素がハッシュ可能であるときのみハッシュ可能となる。ミュータブルな組み込みオブジェクトはすべてハッシュ不可能である。

`object` 型のインスタンスはハッシュ可能である。`object` 型のインスタンス `x` において、`x.__hash__()` は `x == y` が `x is y` と `hash(x) == hash(y)` の両方を意味するような適切な値を返す（`id()` の戻り値から計算している）。ユーザー定義クラスは、`__eq__()` と `__hash__()` メソッドを `object` クラスから継承する。つまり、ユーザー定義クラスでは基本的にインスタンスがハッシュ可能である。しかし、クラス定義に `__hash__ = None` とするオーバーライドを含めると、ハッシュ不能となる。また、`__eq__()` をオーバーライドしていて `__hash__()` を定義していないクラスでは、`__hash__()` は暗黙的に `None` に設定され、ハッシュ不能となる。

In [ ]:
class C1:
    def __init__(self, val):
        self.val = val

    def __eq__(self, other):
        if isinstance(other, C1):
            return self.val == other.val
        else:
            return NotImplemented


class C2:
    def __init__(self, val):
        self.val = val

    def __eq__(self, other):
        if isinstance(other, C2):
            return self.val == other.val
        else:
            return NotImplemented

    def __hash__(self):
        return super().__hash__()


c1 = C1(1)
assert c1.__hash__ is None  # __hash__ 属性が自動的に None で定義される
c2 = C2(2)
d2 = {c2: 2}  # c2 はハッシュ可能なので辞書のキーに使用できる

真理値判定
----------

Python で式やオブジェクトに対して真理値を判定するために評価される場面を**真理値コンテキスト**（truth value context）という。代表的な真理値コンテキストは以下の通り。

  * **if 文の条件**: 式を記述できる（セイウチ演算子を持つ代入式も可）。
  * **while 文の条件**: 式を記述できる（セイウチ演算子を持つ代入式も可）。
  * **if…else 演算の被演算子（オペランド）**: ブール式を記述できる。
  * **ブール演算の被演算子（オペランド）**: ブール式を記述できる。
  * **組み込み関数 bool() の引数**: ブール式を記述できる。

どの真理値コンテキストでも、アトムとしてオブジェクトを記述できる。オブジェクトが真理値コンテキストで評価されるとき、Python は次のルール（**真理値判定のプロトコル**）に従う：

  1. オブジェクトに `__bool__()` メソッドがあれば、それを呼び出す（戻り値は `True` か `False` のどちらかでなければならない）
  2. `__bool__()` が定義されていなければ、`__len__()` メソッドを呼び出し、長さが 0 なら `False`、それ以外は `True`
  3. どちらも定義されていなければ、常に `True`

たとえば、次は全て `False` と判定され、値としてこれらを返す式は `False` と判定される。

　`None`, `False`, `0`, `0.0`, `0j`, `''`, `()`, `[]`, `{}`, `set()`, `range(0)`

これらのオブジェクトは、暗黙の型変換（いわゆる型キャスト）によって `bool` 型に変換されるのではなく、真理値判定のプロトコルによって動的に真理値が判定されることに注意する。

以下は、PEP 8 による推奨事項。

シーケンス（文字列、リスト、タプルなど）は `__len__()` メソッドを持つオブジェクトである。`__len__()` はシーケンスの長さを返し、組み込み関数 `len()` によって呼び出される。シーケンスをブール式として使う場合、空のシーケンスが `False` であることを利用できる。この場合に `len()` を実行して長さを調べるのは画蛇添足。

``` python
# 正しい:
if not seq:
if seq:

# 間違い:
if len(seq):
if not len(seq):
```

ブール型の値と `True` や `False` を比較するのに `==` や `is` を使うべきではない。ブール型オブジェクト自体を式として使えば十分なのであって、`==` 演算子や `is` 演算子を加えるのは画蛇添足。

``` python
# 正しい:
if is_file:

# 間違い:
if is_file == True:
if is_file is True:
```

スライスオブジェクト
--------------------

シーケンスのインデックス参照は、添字表記から組み込みの `slice` オブジェクトを作成して `__getitem__()` メソッドに渡すという処理をしている。添字表記の中に直接 `slice` オブジェクトを指定することもできる。

`slice` のコンストラクタは、引数の数が異なるもので 2 つ存在する:

``` python
slice(stop)
slice(start, stop, step=None)
```

これらは、`range(start, stop, step)` で指定されたインデックスのセットを表す `slice` オブジェクトを返す。

`slice` オブジェクトは、読み出し専用の `start`, `stop`, `step` 属性を持つ。これらは単にコンストラクタ引数の値（またはそのデフォルト値）を返す。`start` と `step` のデフォルト値は `None`。

添字表記に対応する `slice` オブジェクトの属性は、次のとおり。

| 添字表記 | `start` | `stop` | `step` |
|:---|:---|:---|:---|
| `[i:j]` | `i` | `j` | `None` |
| `[i:j:k]` | `i` | `j` | `k` |
| `[i:]` | `i` | `None` | `None` |
| `[:j]` |`None` | `j` | `None` |
| `[::k]` |`None` | `None` | `k` |

In [ ]:
a = [x for x in range(10)]
a[slice(2, 8)], a[slice(None, None, -1)]

([2, 3, 4, 5, 6, 7], [9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

動的型付けとダックタイピング
----------------------------

型に関するエラーの検証を、コンパイル時に行う型システムを**静的型付け**（static typing）、実行時に行う型システムを**動的型付け**（dynamic typing）という。静的型付け言語では、変数や、関数の引数や戻り値が特定のデータ型にバインドされる――一般には宣言に型を記述するが、コンパイラの型推論を利用して型の記述が省略可能な言語もある。一方、動的型付け言語では、データ型はあるが、変数や、関数の引数や戻り値がどのデータ型にもバインドされない。

Python は動的型付け言語である。実行時でないと、変数や引数などに束縛されるオブジェクトに型による制約がないため、「このオブジェクトは〇〇型であるから〇〇メソッドを呼び出せる」というようにコードを書けない。本当の型がわからないままにオブジェクトのメソッドや属性が使われる。こうしたプログラミングスタイルを**ダックタイピング**（duck typing）という。つまり、「アヒルのように歩き、アヒルのように鳴くのなら、それはアヒルである」というスタイルである。

ダックタイピングでは、よく EAFP アプローチが取られる。EAFP はマーフィーの法則「事前に承認を得るより、あとで謝ったほうが簡単である（easier to ask for forgiveness than permission）」の略語である。Python のコードでは、if 文の条件式に `type()` や `isinstance()` の判定を使って例外が発生する要因を事前に取り除くことをせずに、例外が発生してから対処するという方針で try - except 文が使われる。

組み込み関数 `hasattr()` も、EAFP アプローチを取っている。

``` python
hasattr(object, name)
```

この関数は、`object` が持つメソッドや属性の名前が `name` と一致する場合 `True` を返し、そうでない場合 `False` を返すのであるが、実のところ、組み込み関数 `getattr(object, name)` を呼び出して `AttributeError` を送出するかどうかを見ることで実装されている。条件式に `hasattr()` の判定を使った if 文

``` python
if hasattr(object, name):
    object.name()
    ...
else:
    ...
```

は、メソッドが存在するものと仮定し、その仮定が誤っていた場合に例外を捕捉する try - except 文

``` python
try:
    object.name()
except AttributeError:
    ...
except:
    raise
else:
    ...
```

を if 文に見せかけているにすぎない。

遅延属性
--------

インスタンス作成時には属性が存在せず、実際に必要になった時に属性が属性辞書に追加されるような属性アクセスを**遅延アクセス**という。遅延アクセスされる属性を**遅延属性**（lazy attribute）と呼ぶ。

遅延アクセスを実現するには、特殊メソッド `__getattr__(self, name)` を使う。ドット演算子 `.` や組み込み関数 `getattr()` などを使って属性の取得、設定、削除を行う際には、`__getattribute__()` が無条件に呼び出されるが、属性が見つからない場合に、もし `__getattr__()` が定義されているならこれが呼び出される。

In [ ]:
class LazyGetattr:
    def __getattr__(self, name):
        print("属性を追加しました")
        setattr(self, name, 0)  # インスタンス属性辞書に追加
        return getattr(self, name)

data = LazyGetattr()
assert data.__dict__== {}
# 遅延アクセス
print(f"{data.foo=}")
assert data.__dict__== {'foo': 0}
# 属性が追加された後は __getattr__() は呼び出されない
print(f"{data.foo=}")

属性を追加しました
data.foo=0
data.foo=0


遅延属性を使用することで、計算コストが高い属性の評価を必要な時まで遅延させることができる。これにより、プログラムのパフォーマンスが向上する場合がある。

一方、動的に属性が追加されるので、エディタの入力支援が得られない、静的解析ができないというデメリットがある。コードの複雑さが増し、属性の依存関係の管理が難しくなることがある。

シングルトンパターン
--------------------

クラス設計における定石集を**デザインパターン**（design pattern）と呼ぶ。GoF（Gang of Four; 4 人組）による書籍『オブジェクト指向における再利用のためのデザインパターン』の中で取り上げた 23 種類の設計パターンが有名である。ただし、これらの設計パターンは Python 向けのものではないため、Python ではあまり使われないものや、Python の言語仕様として組み込まれているものがある。

**シングルトンパターン**（singleton pattern）は、デザインパターンの 1 つで、そのクラスのインスタンスが 1 つしか生成されないことを保証する設計パターンである。そのようにインスタンス化が制限されるクラスのことを**シングルトンクラス**と呼び、シングルトンクラスの唯一のインスタンスを**シングルトン**と呼ぶ。

Python では、`None`、`NotImplemented`、`Ellipsis` が組み込みのシングルトンであるが、これらは値を変更できない特殊なシングルトンである。

In [ ]:
assert type(None)() is None
assert type(NotImplemented)() is NotImplemented
assert type(Ellipsis)() is Ellipsis

次のコードは、ユーザー定義のシングルトンの例である。

In [ ]:
class Singleton:
    def __new__(cls, *args, **kwargs):
        if not hasattr(cls, "_instance"):
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, input=0):
        self.input = input


if __name__ == "__main__":
    A = Singleton(1)
    B = Singleton(2)
    assert A is B
    assert A.input == B.input == 2

`Singleton` クラスは `__new__()` メソッドを定義しているが、これは `object.__new__()` メソッドをオーバーライドしていることに注意する。`__new__()` の中では、自身のインスタンスを保持する内部クラス変数 `_instance` がなければインスタンスを格納する。以後、`Singleton` のコンストラクタは `_instance` が保持するインスタンスのみを返す。インスタンス作成のために使う `super().__new__(cls)` メソッドは、今は多重継承していないので `object.__new__(cls)` と同一である。残りの可変長引数 `args` と `kwargs` はクラスのコンストラクタ式に渡され、さらに `__init__()` を呼び出す際にその引数に渡される。

シングルトンクラスのインスタンスはアプリケーション全体で共有するので、シングルトンパターンはアプリケーションのグローバルな状態を管理するのに適する。このことは利点である反面、テストを困難にするという欠点にもなる。プログラムの各クラスに対して独立に行うべきテストまでがシングルトンの状態を共有することになるため、前のテストで変更したシングルトンの状態が、次のテストで引き継がれるということが起こってしまう。

サブクラス定義の検証
--------------------

Python のクラス定義は実行可能な文であるから、定義実行時に定義が適切であるかどうかを検証する仕組みを作ることができる。サブクラスの定義を検証する方法は 2 通りある。

1 つ目の方法は、メタクラスを使用することである。次のコード例は、`Polygon` クラスのサブクラスに対して、クラス属性 `sides` が 3 以上の値で初期化されていることを検証するために、`ValidatePolygon` を `Polygon` クラスのメタクラスとしている。基底クラスは暗黙的に `object` クラスのサブクラスになるが、`__new__()` の `bases` 引数は空のタプル `()` が渡されることに注意する。

In [ ]:
class ValidatePolygon(type):
    """多角形を検証するメタクラス"""

    def __new__(cls, name, bases, namespace, **kwargs):
        if bases:  # 基底クラスは検証しない
            if not (sides := namespace.get("sides", 0)):
                raise NotImplementedError("'sides' class attribute not set properly")
            if sides < 3:  # 3未満ならエラー
                raise ValueError("Pylygon need 3 sides")

        return super().__new__(cls, name, bases, namespace, **kwargs)


class Polygon(metaclass=ValidatePolygon):
    pass


class Triangle(Polygon):
    sides = 3


# 以下をコメントアウトすると ValueError が発生する
# class Line(Polygon):
#     sides = 2

2 つ目の方法は、特殊クラスメソッド `__init_subclass__(cls, **kwargs)` を使用することである。クラスが他のクラスを継承するときに必ず親クラスの `__init_subclass__()` が呼び出される。このメソッドは特別扱いされ、自動的にクラスメソッドとなる。第 1 引数 `cls` は新しいサブクラスである。このメソッドを利用すると、サブクラスを定義した際に必ず実行するべき付随操作を書くことができる。上記のコード例で書いた検証を `__init_subclass__()` の中に書いたコードは、次のようになる。

In [ ]:
class Polygon:
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        if "sides" not in cls.__dict__:
            raise NotImplementedError("'sides' class attribute not set properly")
        if getattr(cls, "sides") < 3:  # 3未満なら例外
            raise ValueError("Pylygon need 3 sides")


class Triangle(Polygon):
    sides = 3


# 以下をコメントアウトすると ValueError が発生する
# class Line(Polygon):
#     sides = 2

class 文のヘッダー `class SubClass(BaseClass, foo=value)` に書いたキーワード引数 `foo` は、親クラスの `__init_subclass__()` のキーワード専用引数に引き継がれる。親クラスでは `def __init_subclass__(cls, /, foo, **kwargs):` と定義する。

`object.__init_subclass__(cls)` は何も行わないが、何らかの引数とともに呼び出された場合は、エラーを送出するので注意する。